In [1]:
import pdf2image, os, pytesseract
from img2table.ocr import TesseractOCR
from img2table.document import Image
from datetime import datetime

In [2]:
class IEP_PDF_Reader:
    def __init__(self, pdf_path: str=None) -> None:
        pdf_path = pdf_path if pdf_path else input('Please Provide IEP PDF File Path: ')
        self.pdf_name, pdf_extension = os.path.splitext(pdf_path)
        if pdf_extension != '.pdf': raise Exception('Provided Fike Path Not PDF')
        self.pdf_path = pdf_path
        self.img_pages = pdf2image.convert_from_path(self.pdf_path)
        self.total_page_num = len(self.img_pages)
        self.dir_name = f'{self.pdf_path}_{datetime.now().strftime("%d/%m/%Y %H:%M:%S")}_{self.total_page_num}pgs'
    
    # Reads Text At Given Page Number (Count starts at Page 1)
    def read_page_text(self, page_num) -> str:
        return pytesseract.image_to_string(self._get_page_img_at(page_num))

    # Reads Table At Given Page Number (Count starts at Page 1)
    def read_page_table(self, page_num) -> list:
        return Image(self._get_page_img_at(page_num)).extract_table(ocr=TesseractOCR(n_threads=1, lang='eng'),
                                                implicit_rows=True,
                                                borderless_tables=False,
                                                min_confidence=50)
    
    def read_all(self) -> None:
        for page_num in range(1, self.total_page_num+1):
            file_name = f'{self.dir_name}/p{page_num}_ocr.txt'
            text = f'{self.read_page_text(page_num)}'
            with open(file_name, 'w') as f:
                f.write(text)
    
    # Gets Image At Given Page Number (Count starts at Page 1)
    def _get_page_img_at(self, page_num):
        return self.img_pages[page_num+1]
    


In [3]:
IEP_PDF_Reader("iep.pdf").read_all()

FileNotFoundError: [Errno 2] No such file or directory: 'iep.pdf_11/10/2023 12:00:09_11pgs/p1_ocr.txt'